In [1]:
from rosemary import jpt_setup; jpt_setup()

import argparse
from concurrent.futures import ProcessPoolExecutor
import json
import transformers
from tqdm import tqdm

from rosemary import jpt_parse_args

/gpfs/u/scratch/PTFM/PTFMqngp/minoconda3_x86/envs/open-instruct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

parser = argparse.ArgumentParser()
parser.add_argument("--in-files", nargs="+", type=str)
parser.add_argument("--out-file", type=str, default="sharegpt_split.json")
parser.add_argument("--begin", type=int)
parser.add_argument("--end", type=int)
parser.add_argument("--model-name-or-path", type=str, required=True)
parser.add_argument("--max-length", type=int, default=2048)
# args = parser.parse_args()

cmd = """
--in-files ../data/raw_train/sharegpt/sg_90k_part1_html_cleaned.json ../data/raw_train/sharegpt/sg_90k_part2_html_cleaned.json
--out-file ../data/raw_train/sharegpt/sharegpt_html_cleaned_and_split.json
--model-name-or-path ../results/baselines/huggyllama/llama-7b
"""


cmd = """
--in-files HuggingFaceH4/ultrachat_200k
--out-file ../data/raw_train/ultrachat/ultrachat_200k_splitlongconv.json
--model-name-or-path ../results/baselines/huggyllama/llama-7b
"""

"""
python scripts/split_sharegpt_conversations.py \
    --in-files HuggingFaceH4/ultrachat_200k \
    --out-file data/raw_train/ultrachat/ultrachat_200k_splitlongconv.json \
    --model-name-or-path results/baselines/huggyllama/llama-7b \
    --max-length 2048
"""


# --model-name-or-path huggyllama/llama-7b
# 

args = jpt_parse_args(parser, cmd=cmd)
args

Namespace(in_files=['HuggingFaceH4/ultrachat_200k'], out_file='../data/raw_train/ultrachat/ultrachat_200k_splitlongconv.json', begin=None, end=None, model_name_or_path='../results/baselines/huggyllama/llama-7b', max_length=2048)

In [7]:

p = 'data/processed/ultrachat/ultrachat_data.jsonl'
p = 'data/processed/lima/lima_data.jsonl'

oo = []
with open(p, 'rb') as f:
    for l in f:
        oo.append(json.loads(l))
    
    

    

In [48]:
from datasets import load_dataset
p = 'data/processed/ultrachat/ultrachat_data.jsonl'
# p = 'data/processed/lima/lima_data.jsonl'

ds = load_dataset('json', data_files={'train': p}, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-4fd22f12db15a445/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


{'dataset': 'ultrachat',
 'id': 'ultrachat_0',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
   'role': 'user'},
  {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
   'role': 'assistant'},
  {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collect

In [37]:
import os
output_dir = 'data/processed/ultrachat'
data_dir = 'data/raw_train/ultrachat'
from datasets import load_dataset
output_path = os.path.join(output_dir, 'ultrachat_data.jsonl')
# ds = load_dataset('HuggingFaceH4/ultrachat_200k', cache_dir=data_dir, split='train_sft')
ds = load_dataset('json', data_files={'train': os.path.join(data_dir, 'ultrachat_200k_splitlongconv.json')}, split='train', cache_dir=data_dir, keep_in_memory=True)
# splitting conversations removes "prompt" already.
ds = ds.remove_columns(["prompt_id"])
def add_metadata_fn(example, idx):
    example.update({'dataset': 'ultrachat', 'id': f'ultrachat_{idx}'})
    return example
ds = ds.map(add_metadata_fn, 
            remove_columns=['prompt_id'], # splitting conversations removes "prompt" already.
            with_indices=True, 
            num_proc=10,
            keep_in_memory=True)
# re-ordering the features
ds.select_columns(['dataset', 'id']).add_column('messages', ds['messages'])
ds.to_json(output_path)

Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/data/raw_train/ultrachat/json/default-0defb8b02736b11e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [38]:


ds = ds.select([0]).map(add_metadata_fn, 
            with_indices=True,
            num_proc=10,
            keep_in_memory=True)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [43]:
# my_data[“train”] = my_data[“train”].add_column(“audio”,temp_data[‘audio’])
# 



{'dataset': 'ultrachat',
 'id': 'ultrachat_0',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
   'role': 'user'},
  {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
   'role': 'assistant'},
  {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collect

In [5]:
if args.in_files == ['HuggingFaceH4/ultrachat_200k']:
    data_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data/raw_train/ultrachat'
    from datasets import load_dataset
    ds = load_dataset('HuggingFaceH4/ultrachat_200k', cache_dir=data_dir, split='train_sft')
    ## wpq: convert the formatting to use the split conversation code.
    def convert_message_to_sharegpt_chat_format(m):
        return {'from': 'gpt' if m['role']=='assistant' else 'human',
                'value': m['content']}
    def convert_conversations_to_sharegpt_chat_format(conv):
        return [convert_message_to_sharegpt_chat_format(m) for m in conv]
    prompt_id = ds['prompt_id']
    conversations = ds['messages']
    conversations = [convert_conversations_to_sharegpt_chat_format(x)
                    for x in conversations]
    assert(len(conversations)==len(prompt_id))
    content = [{'id': pid, 'conversations': conv} for pid, conv in zip(prompt_id, conversations)]
else:
    content = []
    for file in args.in_files:
        with open(file, 'rb') as f:
            content.extend(json.load(f))

    
ds

Found cached dataset parquet (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data/raw_train/ultrachat/HuggingFaceH4___parquet/HuggingFaceH4--ultrachat_200k-75c6e299e27d1db5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})

In [6]:


print(len(content))
print(content[0].keys())

i = 1
conversations = content[i]['conversations']
print(len(conversations))
print(conversations[i].keys())

N = 3

for i in range(N):
    print(f"\n===== {conversations[i]['from']} =====")
    print(conversations[i]['value'])

207865
dict_keys(['id', 'conversations'])
12
dict_keys(['from', 'value'])

===== human =====
Which famous landmarks should I visit in London, beyond the usual ones?

===== gpt =====
1. Leadenhall Market - a beautiful indoor market with stunning Victorian architecture, also used as a filming location in the Harry Potter films.

2. St. Dunstan in the East - a ruined church in the middle of the city that has been turned into a beautiful public garden.

3. The Monument - a 202-foot-tall column commemorating the Great Fire of London, with a staircase leading to a viewing platform offering great views of the city.

4. The Camden Town Markets - an eclectic collection of markets offering food, fashion, and vintage items, plus live music and street performers.

5. Novelist's House - the former home of Charles Dickens, now a museum dedicated to his life and works.

6. The Old Operating Theatre - a museum housed in the oldest surviving operating theatre in Europe, with exhibits on the history of 

In [7]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name_or_path,
#     padding_side="right",
    use_fast=False,
)

# mpt-7b

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [31]:
from split_sharegpt_conversations import split_all, filter_invalid_roles

args.end=1000

new_content = split_all(content, args.begin, args.end, tokenizer, args.max_length)

print(f"after split:  {len(new_content)}")
# len(new_content):
# llama:  178604
# mpt-7b: 168815
#
new_content = filter_invalid_roles(new_content)

print(f"after filter: {len(new_content)}")
# len(new_content):
# mpt-7b: 160564


  0%|          | 3/1000 [00:01<09:30,  1.75it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2093 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2578 > 2048). Running this sequence through the model will result in indexing errors
100%|██████████| 1000/1000 [00:02<00:00, 407.94it/s]


after split:  1057
after filter: 1051


In [32]:

if args.in_files == ['HuggingFaceH4/ultrachat_200k']:
    ## wpq: convert back to ultrachat format
    def convert_messages_to_ultrachat_format(m):
        return {'content': m['value'], 
                'role': 'user' if m['from']=='human' else 'assistant'}
    def convert_conversations_to_ultrachat_format(conv):
        return [convert_messages_to_ultrachat_format(m) for m in conv] 
    new_content = [{'prompt_id': x['id'], 
    'conversations': convert_conversations_to_ultrachat_format(x['conversations'])} 
        for x in new_content]

In [46]:
from collections import Counter
import numpy as np

l = content[:1000]
nconv = [len(x['conversations']) for x in l]
print(dict(Counter(nconv)))
ntokens = [len(tokenizer(''.join([y['value'] for y in x['conversations']])).input_ids) for x in l]
print(np.max(ntokens))


Token indices sequence length is longer than the specified maximum sequence length for this model (3571 > 2048). Running this sequence through the model will result in indexing errors


{8: 265, 12: 19, 4: 336, 6: 334, 14: 13, 10: 33}
4631


In [47]:
l = new_content[:1000]
nconv = [len(x['conversations']) for x in l]
print(dict(Counter(nconv)))
ntokens = [len(tokenizer(''.join([y['content'] for y in x['conversations']])).input_ids) for x in l]
print(np.max(ntokens))

{8: 175, 12: 14, 2: 38, 4: 392, 6: 345, 10: 30, 14: 6}
2018


In [23]:

print(f"total: {len(content)}, new: {len(new_content)}")
# json.dump(new_content, open(args.out_file, "w"), indent=2)


total: 207865, new: 1057


In [23]:
from split_sharegpt_conversations import split_one_sample

sample = content[0]

new_sample = split_one_sample(sample)

print(len(sample['conversations']), len(new_sample))


38 5


In [22]:
# def split_one_sample(sample):


tokenized_lens = []
conversations = sample["conversations"]
conversations = conversations[: len(conversations) // 2 * 2]
for c in conversations:
    length = len(tokenizer(c["value"]).input_ids) + 6
    tokenized_lens.append(length)

start_idx = 0
cur_len = 0

if len(conversations) % 2 != 0 or len(conversations) < 2:
    return []






[{'id': 'Og9h3C1_0',
  'conversations': [{'from': 'human',
    'value': "root@openvpn:/home/openvpn# ./openvpn-install.sh\nWelcome to OpenVPN-install!\nThe git repository is available at: https://github.com/angristan/openvpn-install\n\nIt looks like OpenVPN is already installed.\n\nWhat do you want to do?\n 1) Add a new user\n 2) Revoke existing user\n 3) Remove OpenVPN\n 4) Exit\nSelect an option [1-4]: 1\n\nTell me a name for the client.\nThe name must consist of alphanumeric character. It may also include an underscore or a dash.\nClient name: naam\n\nDo you want to protect the configuration file with a password?\n(e.g. encrypt the private key with a password)\n 1) Add a passwordless client\n 2) Use a password for the client\nSelect an option [1-2]: 1\n\nNote: using Easy-RSA configuration from: /etc/openvpn/easy-rsa/vars\nUsing SSL: openssl OpenSSL 3.0.2 15 Mar 2022 (Library: OpenSSL 3.0.2 15 Mar 2022)\n-----\nUsing configuration from /etc/openvpn/easy-rsa/pki/easy-rsa-54848.BT2FXv/